In [5]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [6]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [7]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20220925'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [8]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [9]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,6328d51d7ff9294e27562cc5,NaN,8861892c9dfffff,13.042900,77.591309,Bangalore,8861892c9dfffff,13.042987,77.591393,021621,02,0215,Link,02:00,2,20220920,customerCancelled,order cancelled before rider accepted
1,6328dc4105e89e76c166c3b1,1.578,88618925e9fffff,12.942283,77.628036,Bangalore,88618924e7fffff,12.904710,77.590881,024649,02,0245,Link,02:59,2,20220920,dropped,None
2,6328d41b05e89e76c166c1eb,0.991,8861892093fffff,12.933118,77.689003,Bangalore,8861892093fffff,12.933093,77.688972,021203,02,0200,Link,02:00,2,20220920,dropped,None
3,6328dc087d42681b1c448745,NaN,8860145a15fffff,12.942168,77.525932,Bangalore,8860145a15fffff,12.942168,77.525932,024552,02,0245,Link,02:59,2,20220920,customerCancelled,order cancelled before rider accepted
4,6328d65d853c020ac7961395,0.000,88601459edfffff,13.051909,77.513405,Bangalore,88601459edfffff,13.051926,77.513405,022141,02,0215,Link,02:00,2,20220920,dropped,None


In [10]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date
0,6328d51d7ff9294e27562cc5,NaN,8861892c9dfffff,13.042900,77.591309,Bangalore,8861892c9dfffff,13.042987,77.591393,021621,02,0215,Link,02:00,2,20220920,customerCancelled,order cancelled before rider accepted,21,16,20
1,6328dc4105e89e76c166c3b1,1.578,88618925e9fffff,12.942283,77.628036,Bangalore,88618924e7fffff,12.904710,77.590881,024649,02,0245,Link,02:59,2,20220920,dropped,None,49,46,20
2,6328d41b05e89e76c166c1eb,0.991,8861892093fffff,12.933118,77.689003,Bangalore,8861892093fffff,12.933093,77.688972,021203,02,0200,Link,02:00,2,20220920,dropped,None,03,12,20
3,6328dc087d42681b1c448745,NaN,8860145a15fffff,12.942168,77.525932,Bangalore,8860145a15fffff,12.942168,77.525932,024552,02,0245,Link,02:59,2,20220920,customerCancelled,order cancelled before rider accepted,52,45,20
4,6328d65d853c020ac7961395,0.000,88601459edfffff,13.051909,77.513405,Bangalore,88601459edfffff,13.051926,77.513405,022141,02,0215,Link,02:00,2,20220920,dropped,None,41,21,20


In [11]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count
0,6328d51d7ff9294e27562cc5,NaN,8861892c9dfffff,13.042900,77.591309,Bangalore,8861892c9dfffff,13.042987,77.591393,021621,02,0215,Link,02:00,2,20220920,customerCancelled,order cancelled before rider accepted,21.0,16.0,20,20-40,1
1,6328dc4105e89e76c166c3b1,1.578,88618925e9fffff,12.942283,77.628036,Bangalore,88618924e7fffff,12.904710,77.590881,024649,02,0245,Link,02:59,2,20220920,dropped,None,49.0,46.0,20,40-60,1
2,6328d41b05e89e76c166c1eb,0.991,8861892093fffff,12.933118,77.689003,Bangalore,8861892093fffff,12.933093,77.688972,021203,02,0200,Link,02:00,2,20220920,dropped,None,3.0,12.0,20,0-20,1
3,6328dc087d42681b1c448745,NaN,8860145a15fffff,12.942168,77.525932,Bangalore,8860145a15fffff,12.942168,77.525932,024552,02,0245,Link,02:59,2,20220920,customerCancelled,order cancelled before rider accepted,52.0,45.0,20,40-60,1
4,6328d65d853c020ac7961395,0.000,88601459edfffff,13.051909,77.513405,Bangalore,88601459edfffff,13.051926,77.513405,022141,02,0215,Link,02:00,2,20220920,dropped,None,41.0,21.0,20,40-60,1


In [12]:
#Check the number of rides coming from each hex
df_grouped = df.groupby(['customer_location_hex_8'])['count'].sum().reset_index()

#Filter out those hexes where atleast 1000 rides have come in the last two weeks
df_grouped = df_grouped[df_grouped['count'] >= 100].reset_index(drop = True)

#Add a column to keep for analysis
df_grouped['keep_for_analysis'] = 'yes'
df_grouped.head()

,customer_location_hex_8,count,keep_for_analysis
0,883cf2c64bfffff,133,yes
1,8860144b4dfffff,293,yes
2,886014510dfffff,124,yes
3,8860145121fffff,350,yes
4,8860145123fffff,328,yes


In [13]:
#Merge the number of rides to the original df
df = df.merge(df_grouped[['customer_location_hex_8', 'keep_for_analysis']], on = 'customer_location_hex_8', how = 'left')
df['keep_for_analysis'] = df['keep_for_analysis'].fillna('no')
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count,keep_for_analysis
0,6328d51d7ff9294e27562cc5,NaN,8861892c9dfffff,13.042900,77.591309,Bangalore,8861892c9dfffff,13.042987,77.591393,021621,02,0215,Link,02:00,2,20220920,customerCancelled,order cancelled before rider accepted,21.0,16.0,20,20-40,1,yes
1,6328dc4105e89e76c166c3b1,1.578,88618925e9fffff,12.942283,77.628036,Bangalore,88618924e7fffff,12.904710,77.590881,024649,02,0245,Link,02:59,2,20220920,dropped,None,49.0,46.0,20,40-60,1,yes
2,6328d41b05e89e76c166c1eb,0.991,8861892093fffff,12.933118,77.689003,Bangalore,8861892093fffff,12.933093,77.688972,021203,02,0200,Link,02:00,2,20220920,dropped,None,3.0,12.0,20,0-20,1,yes
3,6328dc087d42681b1c448745,NaN,8860145a15fffff,12.942168,77.525932,Bangalore,8860145a15fffff,12.942168,77.525932,024552,02,0245,Link,02:59,2,20220920,customerCancelled,order cancelled before rider accepted,52.0,45.0,20,40-60,1,yes
4,6328d65d853c020ac7961395,0.000,88601459edfffff,13.051909,77.513405,Bangalore,88601459edfffff,13.051926,77.513405,022141,02,0215,Link,02:00,2,20220920,dropped,None,41.0,21.0,20,40-60,1,yes


In [14]:
#Filter the df only on hexes to keep
df_filter = df[df['keep_for_analysis'] == 'yes']

In [15]:
#Groupby on hex to see number of orders coming in 15 and 20 second batches
df_hex_grouped = df_filter.groupby(['customer_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])['count'].sum().reset_index()
df_hex_grouped = df_hex_grouped.sort_values(by = ['customer_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])
df_hex_grouped.head()

,customer_location_hex_8,date,hour,minute,20_sec_batch,count
0,883cf2c64bfffff,19,00,0.0,0-20,0
1,883cf2c64bfffff,19,00,0.0,20-40,0
2,883cf2c64bfffff,19,00,0.0,40-60,0
3,883cf2c64bfffff,19,00,1.0,0-20,0
4,883cf2c64bfffff,19,00,1.0,20-40,0


In [16]:
area1_hexes = pd.read_csv('bangalore_location_data_level1_demand_zone1.csv')
area2_hexes = pd.read_csv('bangalore_location_data_level1_demand_zone2.csv')

list_area1_hexes = area1_hexes['customer_location_hex_8'].unique()
list_area2_hexes = area2_hexes['customer_location_hex_8'].unique()

df_hex_grouped['zone_name'] = np.where(df_hex_grouped['customer_location_hex_8'].isin(list_area1_hexes), 'zone1', 
np.where(df_hex_grouped['customer_location_hex_8'].isin(list_area2_hexes), 'zone2', 'rest_of_bangalore'))

In [17]:
#Identify high demand hexes
df_hex_grouped_filter2 = df_hex_grouped[df_hex_grouped['zone_name'] == 'rest_of_bangalore']
df_hex_grouped_filter2 = df_hex_grouped_filter2[df_hex_grouped_filter2['count'] >= 1]
count_df = pd.DataFrame(df_hex_grouped_filter2['customer_location_hex_8'].value_counts().reset_index())
count_df.columns = ['customer_location_hex_8', 'count']
count_df.head(20)

,customer_location_hex_8,count
0,8861892421fffff,5812
1,8861892543fffff,5301
2,8861892541fffff,5219
3,8861892553fffff,5089
4,8860145b55fffff,5018
5,8861892ec3fffff,4994
6,886189242dfffff,4824
7,8861892537fffff,4729
8,8861892423fffff,4616
9,886189242bfffff,4520


In [34]:
#Look at the average demand for each minute
df_hex_grouped['time'] = (pd.to_datetime(df_hex_grouped['hour'].astype(str) + ':' + df_hex_grouped['minute'].astype(int).astype(str), format='%H:%M').dt.time)
df_hex_grouped.head()

,customer_location_hex_8,date,hour,minute,20_sec_batch,count,time
0,883c139149fffff,01,00,0.0,0-20,0,00:00:00
1,883c139149fffff,01,00,0.0,20-40,0,00:00:00
2,883c139149fffff,01,00,0.0,40-60,0,00:00:00
3,883c139149fffff,01,00,1.0,0-20,0,00:01:00
4,883c139149fffff,01,00,1.0,20-40,0,00:01:00


In [1]:
#Filter on the hexes identified as high demand
df_high_demand = pd.read_csv('bangalore_location_data_level1_demand_zone1.csv')
high_demand_hexes = list(df_high_demand['customer_location_hex_8'].unique())
hexes = []
for x in high_demand_hexes:
    hex_ids = h3.k_ring(x, 1)
    hexes.append(hex_ids)

NameError: name 'pd' is not defined

In [46]:
#Get the total number of rides per hour
df_level_1_grouped = round(df_hex_high_demand.groupby(['date', 'hour'])['count'].sum().reset_index(), 0)
df_level_1_grouped = df_level_1_grouped.rename(columns = {'count':'rides_per_hour'})
df_level_1_grouped['rides_per_20_sec'] = df_level_1_grouped['rides_per_hour'].apply(lambda x: round(x/180, 2))
df_level_1_grouped

,date,hour,rides_per_hour,rides_per_20_sec
0,01,00,405,2.25
1,01,01,372,2.07
2,01,02,129,0.72
3,01,03,62,0.34
4,01,04,91,0.51
...,...,...,...,...
667,30,19,2287,12.71
668,30,20,1586,8.81
669,30,21,1006,5.59
670,30,22,644,3.58


In [47]:
df_level_1_grouped.to_csv('demand_bangalore.csv', index = False)

In [19]:
list_area2_hexes

array(['8861892087fffff', '8861892097fffff', '88618920a3fffff',
       '88618920b1fffff', '88618920b9fffff', '88618920bbfffff',
       '88618920bdfffff', '8861892091fffff', '8861892093fffff',
       '8861892095fffff', '88618920a7fffff', '88618920abfffff',
       '88618920b3fffff', '88618920b5fffff', '88618920b7fffff',
       '8861892e49fffff'], dtype=object)